# Étude d'une équation de convection scalaire

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [4]:
%matplotlib inline
import numpy             as np
import numpy.linalg      as alg
import matplotlib.pyplot as plt
import time
#
import pyfvm.mesh  as mesh
import pyfvm.model as model
import pyfvm.field as data
from pyfvm.xnum        import *
from pyfvm.integration import *

On cherche à résoudre l'évolution instationnaire du problème linéaire de convection suivant

$$ \frac{\partial q}{\partial t} + a \frac{\partial q}{\partial x} = 0 $$

pour la quantité transportée $q(x,t)$ et la condition intiale $q_0(x)$ sur le domaine $[0;\ell]$ avec des conditions périodiques. On choisit $\ell=1\rm~m$ et $a=1\rm~m/s$. 


## Définition des maillages, du modèle physique et solution initiales


In [5]:
mesh400 = mesh.unimesh(ncell=400, length=1.)
mesh200 = mesh.unimesh(ncell=200, length=1.)
mesh100 = mesh.unimesh(ncell=100, length=1.)
mesh50  = mesh.unimesh(ncell=50,  length=1.)
mgmesh  = mesh.refinedmesh(ncell=100, length=1., ratio=2.)

mymodel = model.convmodel(1.)
bc = 'p'

# sinus packet
def init_sinpack(mesh):
    return np.sin(2*2*np.pi/mesh.length*mesh.centers())*(1+np.sign(-(mesh.centers()/mesh.length-.25)*(mesh.centers()/mesh.length-.75)))/2        
    
# periodic wave
def init_sinper(mesh):
    k = 2 # nombre d'onde
    return np.sin(2*k*np.pi/mesh.length*mesh.centers())
    
# square signal
def init_square(mesh):
    return (1+np.sign(-(mesh.centers()/mesh.length-.25)*(mesh.centers()/mesh.length-.75)))/2


## Calcul et comparaison de spectre


1. le spectre dépend-t-il de la condition initiale ? pourquoi ?
2. qu'attend-t-on de la partie droite du spectre ? 
3. qu'attend-t-on de la partie gauche du spectre et quel est son impact ?

In [6]:
meshs   = [ mesh100 ]
# extrapol1(), extrapol2()=extrapolk(1), centered=extrapolk(-1), extrapol3=extrapol(1./3.) 
xmeths   = [ extrapol1(), extrapol3() ]
legends  = [ 'O1', 'O3',  ]
# explicit, rk2, rk3ssp, rk4, implicit, trapezoidal=cranknicolson

init = init_sinper

results = []
labels  = []
nbcalc  = max(len(xmeths), len(meshs))

for i in range(nbcalc):
    lmesh = (meshs*nbcalc)[i]
    field0 = scafield(mymodel, bc, lmesh.ncell)
    field0.qdata[0] = init(lmesh)
    solver = implicit(lmesh, (xmeths*nbcalc)[i])
    jac    = solver.calc_jacobian(numfield(field0))
    val, vec = alg.eig(jac)
    results.append(val/lmesh.ncell)

# display and save results to png file
style=['o', 'x', 'D', '*', 'o', 'o']
fig=plt.figure(1, figsize=(10,8))
for i in range(nbcalc):
    plt.scatter(results[i].real, results[i].imag, marker=style[i])
    labels.append(legends[i])
plt.legend(labels, loc='upper left',prop={'size':10})

## Spectre d'opérateur et limite de stabilité

1. Comment évolue l'opérateur d'intégration temporelle (développement de $\exp At$) ?
2. Tracer le diagramme pour différents couples schémas temporels et spatials et retrouver les limites de stabilité par la simulation

In [49]:
meshs   = [ mesh50 ]
# extrapol1(), extrapol2()=extrapolk(1), centered=extrapolk(-1), extrapol3=extrapol(1./3.) 
xmeths   = [ extrapol3() ]
legends  = [ 'O3' ]

init = init_sinper
cfl  = 1.8

results = []
labels  = []
nbcalc  = max(len(xmeths), len(meshs))

for i in range(nbcalc):
    lmesh = (meshs*nbcalc)[i]
    field0 = scafield(mymodel, bc, lmesh.ncell)
    field0.qdata[0] = init(lmesh)
    solver = implicit(lmesh, (xmeths*nbcalc)[i])
    jac    = solver.calc_jacobian(numfield(field0))
    val, vec = alg.eig(jac)
    results.append(val/lmesh.ncell)

# display 
style=['o', 'x', 'D', '*', 'o', 'o']
fig=plt.figure(1, figsize=(10,8))
for i in range(nbcalc):
    plt.scatter(results[i].real, results[i].imag, marker=style[i])
    labels.append(legends[i])
plt.legend(labels, loc='upper left',prop={'size':10})
#
# COMPUTE AND DISPLAY STABILITY AREA
#
x = np.r_[-2.5:.2:30j]
y = np.r_[-2.:2.:60j]
X, Y = np.meshgrid(x, y)
vp = X+Y*1j
#
def integrator(v):
    # A EVENTUELLEMENT COMPLETER
    vt = cfl*v
    return 1.+ vt + .5*vt**2 + 1./6.*vt**3
#
plt.contour(X,Y,abs(integrator(vp)), levels=[1], linewidths=3, colors='darkorange') # contour() accepts complex values
plt.axis('equal')
gain    = abs(integrator(results[i]))
vp_unst = np.where(gain > 1)
imax    = gain.argmax()
print "nombre de valeurs instables:", vp_unst[0].size, results[i][imax],"\ntaux max:",gain[imax]
plt.scatter(results[i][imax].real, results[i][imax].imag, s=100, marker='o', alpha=0.4, color='r')


---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [52]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style